In [1]:
from math import floor
import math
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix, classification_report, make_scorer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
import numpy as np
import common # type: ignore
from sklearn.feature_selection import VarianceThreshold
from mlxtend.feature_selection import SequentialFeatureSelector
from IPython.display import display, Markdown, Latex, HTML
import matplotlib.pyplot as plt
import matplotlib.colors as mpcolors
from sqlalchemy import create_engine, text
import quantiphy as qq
import warnings
import enum
import os
import importlib
import latex
from latex import dm, pp, ptime, Cites, AC
from latex import Tables, Table, Figure, Figures
from latex import is_latex, set_latex, unset_latex
import math

EPS_TH = 0.5
DATASET = "CTU-13"
database = common.Database()
dataset = common.Dataset()

# (DN, ONLYFIRSTS, NO-WHITELISTING)

config = common.SlotConfig(sps=1 * 60 * 60, th=0.999, wl_th=1000_000, dataset=common.DATASETS.CTU13, onlyfirsts='pcap', wl_col=common.WLFIELD.DN)

slots = {}

def get_slot(slots, config):
    if config not in slots:
        slots[config] = common.Slot(database, config)
    return slots[config]

get_slot(slots, config).df




,slotnum,pcap_dga,dga,pcap_infected,pcaps_count,pcaps,rank_bdn,pcaps_count_pos1,pcaps_count_pos2,pcaps_count_pos3,...,pos3_wl,pos4_wl,neg1_dnagg,neg2_dnagg,neg3_dnagg,neg4_dnagg,pos1_dnagg,pos2_dnagg,pos3_dnagg,pos4_dnagg
0,0.0,0,0,False,17,"[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20...",2192,16,13,13,...,0,2,"[30.189.227.61.in-addr.arpa, 250.45.196.98.in-...","[30.189.227.61.in-addr.arpa, 250.45.196.98.in-...","[30.189.227.61.in-addr.arpa, 250.45.196.98.in-...","[30.189.227.61.in-addr.arpa, 250.45.196.98.in-...",[p4-ftrkjptt2hf6o-cqi5iohiogujmodr-if-v6exp3-v...,"[idsync.rlcdn.com, idsync.rlcdn.com, idsync.rl...","[bcp.crwdcntrl.net, sync.rhythmxchange.com, sy...","[dc8xl0ndzn2cb.cloudfront.net, typeface.nyt.co..."
1,0.0,1,1,True,20,"[26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 3...",1258,6,8,5,...,2,3,"[dns.msftncsi.com, yandex.ru, musclecuphospita...","[dns.msftncsi.com, yandex.ru, musclecuphospita...","[dns.msftncsi.com, yandex.ru, musclecuphospita...","[dns.msftncsi.com, yandex.ru, musclecuphospita...","[coopvgg.com.ar, coopvgg.com.ar, www.tognolagr...","[requestpressure.com, idsync.rlcdn.com, idsync...","[requestpressure.com, bcp.crwdcntrl.net, yasou...","[requestpressure.com, substanceissue.com, ads...."
2,0.0,2,2,True,13,"[37, 41, 42, 45, 46, 50, 51, 52, 54, 55, 56, 5...",0,10,10,10,...,0,0,"[cyropumtsolp.xyz, erojrbusyhmi.xyz, oxmijnqux...","[nyotcjmtsrrr.xyz, eursccrjbkhh.xyz, opcotscde...","[opcotscdeedh.xyz, rberspijorqi.xyz, redirslon...","[cyropumtsolp.xyz, rberspijorqi.xyz, etntlrolu...","[evinobxksfeu.xyz, nyotcjmtsrrr.xyz, eursccrjb...","[evinobxksfeu.xyz, jwcvmurslmbc.xyz, eurjmqhre...","[evinobxksfeu.xyz, nyotcjmtsrrr.xyz, eursccrjb...","[evinobxksfeu.xyz, nyotcjmtsrrr.xyz, eursccrjb..."
3,1.0,0,0,False,13,"[10, 11, 13, 14, 15, 16, 17, 18, 20, 21, 22, 2...",1032,9,6,7,...,0,0,"[secure.adnxs.com, aax.amazon-adsystem.com, ad...","[secure.adnxs.com, aax.amazon-adsystem.com, ad...","[secure.adnxs.com, aax.amazon-adsystem.com, ad...","[secure.adnxs.com, aax.amazon-adsystem.com, ad...","[d1p6rqiydn62x8.cloudfront.net, d2p03fkvpgzlmh...",[i2-dckqaeqrldglvfprmtfbrnrqquotiu.init.cedexi...,"[wrxads.golfwrx.com, www.golfwrx.com, 2w6kxc22...","[d1p6rqiydn62x8.cloudfront.net, d3ojqkc95d8mrd..."
4,1.0,1,1,True,12,"[26, 31, 33, 34, 38, 40, 43, 44, 47, 48, 49, 53]",914,6,3,3,...,2,1,"[gmail.com, 299.63525.pdl.alacartebelini.com, ...","[gmail.com, 299.63525.pdl.alacartebelini.com, ...","[gmail.com, 299.63525.pdl.alacartebelini.com, ...","[gmail.com, 299.63525.pdl.alacartebelini.com, ...",[p4-bxt3lwixbdv2g-gmhauo52yz2j2xxg-if-v6exp3-v...,"[mijnbedrijfdraaitdoor.nl, www.uuofscv.org, da...","[www.nimblebuy.com, www.unytygroup.com, songst...","[6512andgrowing.com, fuerihrezaehne.de, songst..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,1008.0,2,2,True,1,[46],0,1,1,1,...,0,0,[],[],[],[],"[tqgmnljrrkeihztrkrsnv.com, onirtjnsweqmzijvpz...","[tqgmnljrrkeihztrkrsnv.com, onirtjnsweqmzijvpz...","[tqgmnljrrkeihztrkrsnv.com, onirtjnsweqmzijvpz...","[tqgmnljrrkeihztrkrsnv.com, onirtjnsweqmzijvpz..."
1449,1009.0,2,2,True,1,[46],0,1,1,1,...,0,0,[],[],[],[],"[uwcejnhetxwhizhfqpveqtddmrg.biz, pcajbnbhmbqn...","[uwcejnhetxwhizhfqpveqtddmrg.biz, pcajbnbhmbqn...","[uwcejnhetxwhizhfqpveqtddmrg.biz, pcajbnbhmbqn...","[uwcejnhetxwhizhfqpveqtddmrg.biz, pcajbnbhmbqn..."
1450,1010.0,2,2,True,1,[46],0,1,1,1,...,0,0,[],[],[],[],"[kfkjdxmfusxohwssgdttg.org, yxknpqoxtkofdmypmb...","[kfkjdxmfusxohwssgdttg.org, yxknpqoxtkofdmypmb...","[kfkjdxmfusxohwssgdttg.org, yxknpqoxtkofdmypmb...","[kfkjdxmfusxohwssgdttg.org, yxknpqoxtkofdmypmb..."
1451,1061.0,1,1,True,1,[32],1,0,0,0,...,0,0,"[accounts.google.com.gh, master.anonymox.net, ...","[accounts.google.com.gh, master.anonymox.net, ...","[accounts.google.com.gh, master.anonymox.net, ...","[accounts.google.com.gh, master.anonymox.net, ...",[],[],[],[]


In [5]:
tt = ["qr", "q","r", "nx"]

b = []
q = "WITH\n"
for t in tt:
    for g in ["pcap", "global"]:
        q += f"""{t}_{g} as (SELECT COUNT(*) as {t}_{g} FROM {t}_{g} WHERE pcap_malware_type='no-malware'),\n"""
        b.append(f"{t}_{g}")

q += """SELECT * FROM """ + ",".join(b)

print(q)

WITH
qr_pcap as (SELECT COUNT(*) as qr_pcap FROM qr_pcap WHERE pcap_malware_type='no-malware'),
qr_global as (SELECT COUNT(*) as qr_global FROM qr_global WHERE pcap_malware_type='no-malware'),
q_pcap as (SELECT COUNT(*) as q_pcap FROM q_pcap WHERE pcap_malware_type='no-malware'),
q_global as (SELECT COUNT(*) as q_global FROM q_global WHERE pcap_malware_type='no-malware'),
r_pcap as (SELECT COUNT(*) as r_pcap FROM r_pcap WHERE pcap_malware_type='no-malware'),
r_global as (SELECT COUNT(*) as r_global FROM r_global WHERE pcap_malware_type='no-malware'),
nx_pcap as (SELECT COUNT(*) as nx_pcap FROM nx_pcap WHERE pcap_malware_type='no-malware'),
nx_global as (SELECT COUNT(*) as nx_global FROM nx_global WHERE pcap_malware_type='no-malware'),
SELECT * FROM qr_pcap,qr_global,q_pcap,q_global,r_pcap,r_global,nx_pcap,nx_global


In [2]:
import common
import latex
importlib.reload(common)
importlib.reload(latex)

def ploo(df, pos_col, tot_col):
    labelneg = f"neg_col"
    labelpos = pos_col
    df = df.groupby(["dga", "slotnum"]).agg({
        tot_col: "sum",
        f"{pos_col}": "sum"
    }).unstack(0).fillna(0)#.reset_index().copy()
    
    for dga in range(3):
        df[labelneg, dga] = df[tot_col, dga] - df[pos_col, dga]
    
    fig, axs = plt.subplots(figsize=(10, 6), tight_layout=True)
    
    W = 1
    ticks_label = []
    ticks_pos = []
    for slotnum, row in df.iterrows():
        if slotnum >= 0 and slotnum <= 10:
            ticks_label.append(slotnum)
            ticks_pos.append(slotnum * 4*W)
            for dga in range(3):
                neg, pos = row[(labelneg, dga)], row[(labelpos, dga)]
                rects = axs.bar(slotnum * 4*W + [-W, 0, W][dga], neg, color=mpcolors.to_rgba(["blue","orange","red"][dga], 0.2))
                rects = axs.bar(slotnum * 4*W + [-W, 0, W][dga], pos, bottom=neg, color=mpcolors.to_rgba(["blue","orange","red"][dga], 1))
                if (pos+neg) > 0:
                    axs.bar_label(rects, labels=[qq.Quantity(int(pos)).render(prec=1)], padding=10)
                axs.bar_label(rects, labels=[qq.Quantity(int(pos+neg)).render(prec=1)], padding=0)
        pass
    
    axs.set_xticks(ticks_pos, ticks_label)
    
    fig1 = Figure(fig,axs, Figures.SLOTS_PCAP, f"({labelneg}, {labelpos})\n{slots[config].config}.")
    # fig1.show()
    
    plt.close()
    return df, fig1

config = common.SlotConfig(sps=10 * 60, th=1 - 0.5, wl_th=0, dataset=common.DATASETS.INFECTED_PLUS_DGA, onlyfirsts='global', wl_col=common.WLFIELD.DN)
df = get_slot(slots, config).df.copy()
df, fig = ploo(df, "pcaps_count_pos1", "pcaps_count")
fig.show()

![](images/slots-pcap/20240711_182038251224.svg "Example")

In [3]:
import common
import latex
importlib.reload(common)
importlib.reload(latex)

dfs={}
for ds in [common.DATASETS.INFECTED_PLUS_DGA, common.DATASETS.CTU13]:
    config = common.SlotConfig(sps=1 * 60 * 60, th=0.5, wl_th=int(10e7), dataset=ds, onlyfirsts='global', wl_col=common.WLFIELD.DN)
    
    dfs[ds], fig = ploo(get_slot(slots, config).df, "pos1", "count")

    # fig.show()
    pass

display(dfs[common.DATASETS.INFECTED_PLUS_DGA] - dfs[common.DATASETS.CTU13])

count           pos1           neg_col          
dga         0    1    2    0    1    2       0    1    2
slotnum                                                 
0.0       0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
1.0       0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
2.0       0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
3.0       0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
4.0       0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
...       ...  ...  ...  ...  ...  ...     ...  ...  ...
1008.0    0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
1009.0    0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
1010.0    0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
1061.0    0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0
1062.0    0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0

[1008 rows x 9 columns]

In [10]:
import common
import latex
import importlib
importlib.reload(common)
importlib.reload(latex)

for i in range(1, 20, 5):
    th = 1 - math.pow(10, -i)
    if th < 1:
        print(f"1-10e-{i}", th)
        config = common.SlotConfig(sps=60 * 60, th=th, wl_th=0, dataset=common.DATASETS.CTU13, packet_type="nx", onlyfirsts="global", wl_col=common.WLFIELD.DN)
        
        df, fig = ploo(get_slot(slots, config).df, "pos1", "count")

        fig.show(to_pdf=False)
        pass
    pass

1-10e-1 0.9


![](images/slots-pcap/20240711_182342672154.svg "Example")

1-10e-6 0.999999


![](images/slots-pcap/20240711_182342800232.svg "Example")

1-10e-11 0.99999999999


![](images/slots-pcap/20240711_182342931831.svg "Example")

1-10e-16 0.9999999999999999


![](images/slots-pcap/20240711_182343063581.svg "Example")

In [11]:
from datetime import datetime, timezone, timedelta

# Provided date and time
date_str = "Sun Jul 26 14:43:52 CEST 2015"

# Define the time zone (CEST is UTC+2)
ce = timezone(timedelta(hours=2))

# Parse the date string
date_time = datetime.strptime(date_str, "%a %b %d %H:%M:%S CEST %Y")

# Assign the correct time zone
date_time = date_time.replace(tzinfo=ce)

# Convert to UTC
date_time_utc = date_time.astimezone(timezone.utc)

# Get the Unix timestamp
timestamp = int(date_time_utc.timestamp())
timestamp

1437914632

In [26]:
df['count',0].sum()



17652.0

In [ ]:
import common
import latex
importlib.reload(common)
importlib.reload(latex)
database.conn.close()

database.conn = database.engine.connect()

config = common.SlotConfig(sps=1 * 60 * 60, th=0.9, wl_th=10e7, dataset=common.DATASETS.CTU13, onlyfirsts="global", wl_col=common.WLFIELD.DN)

df = get_slot(slots, config).slot(slotnum=0, mwtype=common.MWTYPE.DGA)

df

,slotnum,dga,rank_bdn,u,neg1,neg2,neg3,neg4,pos1,pos2,...,wl_pos3,wl_pos4,dnagg_nn1_neg,dnagg_nn2_neg,dnagg_nn3_neg,dnagg_nn4_neg,dnagg_nn1_pos,dnagg_nn2_pos,dnagg_nn3_pos,dnagg_nn4_pos
0,0.0,0,0,8689,8402,8651,8552,8379,287,38,...,0,0,"[ocsp.digicert.com, update.googleapis.com, ter...","[ocsp.digicert.com, update.googleapis.com, ter...","[ocsp.digicert.com, update.googleapis.com, ter...","[ocsp.digicert.com, update.googleapis.com, ter...","[static-cdn.jtvnw.net, tracking-protection.cdn...","[tags.crwdcntrl.net, bcp.crwdcntrl.net, idsync...","[stackoverflow.com, tags.crwdcntrl.net, bcp.cr...","[d3ag4hukkh62yn.cloudfront.net, d1ykf07e75w7ss..."
1,0.0,1,0,9491,8403,8418,8392,8367,1088,1073,...,0,0,"[www.microsoft.com, google.com, sync-tm.everes...","[www.microsoft.com, google.com, sync-tm.everes...","[www.microsoft.com, google.com, sync-tm.everes...","[www.microsoft.com, google.com, sync-tm.everes...","[ciscobinary.openh264.org, d1zkz3k4cclnv6.clou...","[requestpressure.com, substanceissue.com, cloc...","[www.jdoqocy.com, requestpressure.com, substan...","[d1zkz3k4cclnv6.cloudfront.net, statcounter.co..."
2,0.0,2,0,4752,38,53,42,57,4714,4699,...,0,0,"[dns.msftncsi.com, www.msftncsi.com, wpad.lan,...","[dns.msftncsi.com, www.msftncsi.com, wpad.lan,...","[dns.msftncsi.com, www.msftncsi.com, wpad.lan,...","[wpad.lan, fffreenet.xyz, longlexsfsbj.xyz, sl...","[rkpiqcmmmbue.xyz, jxihgmogkkgs.xyz, jjvfxwoks...","[rkpiqcmmmbue.xyz, jxihgmogkkgs.xyz, jjvfxwoks...","[rkpiqcmmmbue.xyz, jxihgmogkkgs.xyz, jjvfxwoks...","[dns.msftncsi.com, www.msftncsi.com, rkpiqcmmm..."


In [ ]:
import common
importlib.reload(common)

display(Markdown(
"""

# Comparision between domain duplicates
                 
Imaging that all the DNS packets generated within the infrastructure flux from
our IDS, we want to analyze the different slot metrics relative to the first
appereance of a domain. We set three possible handlings:

1. Consider new all the domains.
                 
2. Consider new just the domains which appears for the first time within a
   terminal.

3. Consider new just the domains which appears for the first time in all the
   infrastructure.

> Note that in the second and third cases, some captures may appear to have no domain names.
to have no domain names, simply because their domains have already appeared in other
captures.

Given this, as can be seen in the Figures, the number of available domain for
DGA capture:

- in the first case: - The number of domains per slot remains very high for a
  DGA capture. - The number of domains per slot varies slightly for non-DGA
  shots. - The true/negative ratio doesn't change.

- in the second case: - The number of domains per slot drops for all the
  infection states. - The true/negative ratio doesn't change. - The dn/slot drop
  is higher for DGA capture, 
"""))

for of in ["pcap", "global"]:
    config = common.SlotConfig(sps=1 * 60 * 60, th=1 - 0.5, wl_th=0, dataset=common.DATASETS.CTU13, onlyfirsts=of, wl_col=common.WLFIELD.DN)
    df, fig = ploo(get_slot(slots, config).df, "pos1", "count")
    fig.show()
    pass




# Comparision between domain duplicates
                 
Imaging that all the DNS packets generated within the infrastructure flux from
our IDS, we want to analyze the different slot metrics relative to the first
appereance of a domain. We set three possible handlings:

1. Consider new all the domains.
                 
2. Consider new just the domains which appears for the first time within a
   terminal.

3. Consider new just the domains which appears for the first time in all the
   infrastructure.

> Note that in the second and third cases, some captures may appear to have no domain names.
to have no domain names, simply because their domains have already appeared in other
captures.

Given this, as can be seen in the Figures, the number of available domain for
DGA capture:

- in the first case: - The number of domains per slot remains very high for a
  DGA capture. - The number of domains per slot varies slightly for non-DGA
  shots. - The true/negative ratio doesn't change.

- in the second case: - The number of domains per slot drops for all the
  infection states. - The true/negative ratio doesn't change. - The dn/slot drop
  is higher for DGA capture, 


           count                    pos1                
dga            0        1       2      0       1       2
slotnum                                                 
0.0      15690.0  13616.0  6971.0  591.0  1148.0  6909.0
1.0       6376.0   5294.0   317.0  288.0   138.0   315.0
2.0       3489.0   3311.0     7.0  161.0    59.0     7.0
3.0       2770.0   3383.0     4.0  115.0    84.0     4.0
4.0        527.0   3864.0     6.0   10.0    96.0     6.0
...          ...      ...     ...    ...     ...     ...
1008.0       0.0      0.0     4.0    0.0     0.0     4.0
1009.0       0.0      0.0     4.0    0.0     0.0     4.0
1010.0       0.0      0.0     2.0    0.0     0.0     2.0
1061.0       0.0      4.0     0.0    0.0     0.0     0.0
1062.0       0.0      1.0     0.0    0.0     0.0     0.0

[1010 rows x 6 columns]


![](images/slots-pcap/20240701_143754530578.svg "Example")

          count                   pos1                
dga           0       1       2      0       1       2
slotnum                                               
0.0      8689.0  9491.0  4752.0  345.0  1136.0  4721.0
1.0      3622.0  4953.0   275.0  170.0   136.0   273.0
2.0      1846.0  3213.0     7.0   87.0    59.0     7.0
3.0      1552.0  3310.0     4.0   51.0    83.0     4.0
4.0       259.0  3790.0     6.0    6.0    95.0     6.0
...         ...     ...     ...    ...     ...     ...
1008.0      0.0     0.0     4.0    0.0     0.0     4.0
1009.0      0.0     0.0     4.0    0.0     0.0     4.0
1010.0      0.0     0.0     2.0    0.0     0.0     2.0
1061.0      0.0     3.0     0.0    0.0     0.0     0.0
1062.0      0.0     1.0     0.0    0.0     0.0     0.0

[1008 rows x 6 columns]


![](images/slots-pcap/20240701_143755027903.svg "Example")

In [ ]:
import common
importlib.reload(common)

database.conn = database.engine.connect()

slots = {}

for nn in range(1, 5):
    config = common.SlotConfig(sps=1 * 60 * 60, th=1 - 0.5, wl_th=0, dataset=common.DATASETS.CTU13, onlyfirsts="global", wl_col=common.WLFIELD.DN)
    df = get_slot(slots, config).df.copy()
    df, fig = ploo(df, "pcaps_count_pos", "pcaps_count")
    fig.show()

    continue

NameError: name 'ploo' is not defined

        pcaps_count             pcaps_count_pos1           
dga               0     1     2                0    1     2
slotnum                                                    
0.0            17.0  20.0  13.0             16.0  7.0  11.0
1.0            13.0  18.0   8.0             12.0  8.0   8.0
2.0             8.0  15.0   8.0              7.0  5.0   7.0
3.0             7.0  17.0   8.0              7.0  6.0   7.0
4.0             4.0  16.0   8.0              4.0  5.0   7.0
...             ...   ...   ...              ...  ...   ...
1700.0          0.0   1.0   0.0              0.0  0.0   0.0
1701.0          0.0   1.0   0.0              0.0  0.0   0.0
1702.0          0.0   1.0   0.0              0.0  0.0   0.0
1703.0          0.0   1.0   0.0              0.0  0.0   0.0
1704.0          0.0   1.0   0.0              0.0  0.0   0.0

[1705 rows x 6 columns]


![](images/slots-pcap/20240701_142551312422.svg "Example")

In [ ]:
for sn in range(20):
    df = get_slot(slots, config).df.set_index("slotnum")
    print(df[df.dga == 2].loc[sn,"pcaps"])
    print(df[df.dga == 2].loc[sn,"dnagg_nn1_pos"])

[37, 42, 45, 46, 50, 51, 52, 54, 55, 56, 58]
['erqnohygdelo.xyz', 'yghgdmmwceim.xyz', 'yfyyrtxxwnnc.xyz', 'nyedonfmldll.xyz', 'yfskldefoxcc.xyz', 'yfnewfmhengg.xyz', 'sgpumyxsqhtx.xyz', 'shgxmyjudttp.xyz', 'shsfredgfihk.xyz', 'sifspihpxtkr.xyz', 'sjixskhcbidc.xyz', 'yfgtlxwvqpsr.xyz', 'yeelsuuptnvn.xyz', 'ybyvwxijjbig.xyz', 'snilkjfnwwbx.xyz', 'snillsnnelqu.xyz', 'ccliimwmucxh.xyz', 'xxwwxklynogy.xyz', 'xuhotqeyqtsh.xyz', 'snmnowhqjmdj.xyz', 'spdcvfnvdhql.xyz', 'spfpujrbmmnv.xyz', 'spgoixpdmdoo.xyz', 'kwmjnjmylccc.xyz', 'xuddebcddebe.xyz', 'kqhubbbbmfgf.xyz', 'bnmrrwosrcxp.xyz', 'splwyvllyeth.xyz', 'spmnquvsxuve.xyz', 'spoekkrbcndw.xyz', 'spovqpqrmxwd.xyz', 'ibsssjmoqkcb.xyz', 'bnmlpkdhofgh.xyz', 'sppmmbbkrikf.xyz', 'spqreneibrrf.xyz', 'spqrsschyfgy.xyz', 'spqrupcjlyjs.xyz', 'ssoltmxjjvpv.xyz', 'sssffwkjkhgg.xyz', 'sssncdxkjsfb.xyz', 'stktwgnnvcfb.xyz', 'stqrchpfsdri.xyz', 'sxqejyoolyjy.xyz', 'sxtglhqnotqn.xyz', 'sxyegglpnowx.xyz', 'tcxcstvulshh.xyz', 'tdktqgogcsop.xyz', 'tdlqgxppcngx.